In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
sys.path.insert(0, '/home/aew492/CFE_gradients/code')
from generate_mock_list import MockSet
import load_tools
import globals
globals.initialize_vals()

In [4]:
data_dir = globals.data_dir

#### GOAL

We want to write a function that takes in survey parameters and returns the expected gradient recovery.

- $V \rightarrow \sigma$ given fixed tracer density $n$, where $\sigma$ is the precision of recovery
- $V, \gamma \rightarrow \gamma = 0 \pm \mathrm{XX}\sigma$, where XX$\sigma$ is the confidence that $\gamma = 0$

### scaling of $\sigma_\gamma$ with volume $V$

#### simulated results

Since $m$ determines relative gradient, we want to match the ratio $m/L$ so that all the mocks here have the same physical gradient:
$$
\frac{m}{L} = .001
$$

** For now, use what we have, 2D same_omega mocks, but REPLACE THESE with 3D random_omegas mocks to be more robust

In [13]:
Ls = [500, 750, 1000]
ms = [0.5, 0.75, 1.0]
n = '2e-4'
nmocks = 1000
grad_dim = 2
b = 0.5

In [26]:
# recovered gradient amplitudes
grads_rec = np.empty((len(Ls), nmocks, 3))
for i, L in enumerate(Ls):
    mockset = MockSet(L, n, rlzs=nmocks)  # initialize mock set
    mockset.add_gradient(grad_dim, m=ms[i], b=b)  # add gradient
    amps_dict = load_tools.check_grad_amps(mockset, method='suave', plot=False, return_amps=True)
    grads_rec[i] = amps_dict['grads_rec']

In [29]:
# variance in recovered gradient, by component
grads_rec_std = np.std(grads_rec, axis=1)

In [30]:
grads_rec_std.shape

(3, 3)

In [40]:
fig, ax = plt.subplots(figsize=(8,6))

labels = ['x', 'y', 'z']

for i, grad_std in enumerate(grads_rec_std.T):
    ax.plot(Ls, grad_std, marker='.', lw=1, label=labels[i])

ax.axhline(0, color='k', lw=0.5, alpha=0.3)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim((1e-4, 1e-2.5
ax.set_xlabel(r'L ($h^{-1}$Mpc)')
ax.set_ylabel(r'$\sigma_{\gamma, x}$')
ax.set_title(r'Std. in $\gamma_x$ as a function of boxsize $L$')
ax.legend()

SyntaxError: invalid decimal literal (<ipython-input-40-702c6bd24d30>, line 11)

In [5]:
# Roman's equation
# ** predicted variance in the 2pcf at a certain length scale r
def xi_err_pred(r, V, n, k, Pk, nk=1000):

    # interpolate between the discrete (k, Pk) values to get a finer scale for integration
    kfine = np.linspace(min(k), max(k), nk)
    Pkfine = np.interp(kfine, k, Pk)
    
    # multiplicative constant
    const = 1 / (2*np.pi*V)

    # function of k that we want to integrate
    def k_func(k, Pk):
        return (k/r) * (Pk+(1/n))**2 * (special.jv(1/2, k*r))**2

    # construct our array, and integrate using trapezoid rule
    k_func_arr = np.array([k_func(k, Pkfine[i]) for i, k in enumerate(kfine)])
    trapz = integrate.trapz(k_func_arr, x=kfine)

    return const*trapz